In [1]:
from keras import layers
from keras import models
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
train_dir = 'E:/Jupyter NB/10 class Classification/Train_val_test/train'
val_dir = 'E:/Jupyter NB/10 class Classification/Train_val_test/val'
test_dir = 'E:/Jupyter NB/10 class Classification/Train_val_test/test'

In [3]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(299, 299, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 297, 297, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 148, 148, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 146, 146, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 73, 73, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 71, 71, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 35, 35, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 33, 33, 128)      

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(299, 299),
    batch_size=20,
    class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(299, 299),
    batch_size=20,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(299, 299),
    batch_size=20,
    class_mode='categorical')

Found 1233 images belonging to 10 classes.
Found 200 images belonging to 10 classes.
Found 200 images belonging to 10 classes.


# Model Compilation

In [5]:
from keras.losses import categorical_crossentropy as logloss
from keras import backend as K

def loss(y_true, y_pred):
    entropy = -K.mean(K.sum(y_pred*K.log(y_pred), 1))
    beta = 0.1
    return logloss(y_true, y_pred) - beta*entropy

In [6]:
from keras import optimizers

model.compile(loss=loss,
    optimizer=optimizers.SGD(lr=1e-2),
    metrics=['categorical_crossentropy', 'accuracy'])

# Training

In [7]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=15,
    validation_data=validation_generator,
    validation_steps=50)

Epoch 1/15
100/100 [==============================] - 38s 377ms/step - loss: 1.9566 - categorical_crossentropy: 2.1784 - accuracy: 0.2464 - val_loss: 2.1685 - val_categorical_crossentropy: 2.2051 - val_accuracy: 0.2000
Epoch 2/15
100/100 [==============================] - 32s 320ms/step - loss: 1.7534 - categorical_crossentropy: 1.9607 - accuracy: 0.3600 - val_loss: 2.0154 - val_categorical_crossentropy: 2.0602 - val_accuracy: 0.2200
Epoch 3/15
100/100 [==============================] - 32s 320ms/step - loss: 1.5456 - categorical_crossentropy: 1.7311 - accuracy: 0.4426 - val_loss: 1.8416 - val_categorical_crossentropy: 1.8096 - val_accuracy: 0.3450
Epoch 4/15
100/100 [==============================] - 32s 321ms/step - loss: 1.3957 - categorical_crossentropy: 1.5686 - accuracy: 0.4862 - val_loss: 1.7030 - val_categorical_crossentropy: 1.7601 - val_accuracy: 0.4050
Epoch 5/15
100/100 [==============================] - 32s 319ms/step - loss: 1.2911 - categorical_crossentropy: 1.4555 - acc

In [12]:
model.save_weights('teacher_weights.hdf5')

In [13]:
'''
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)
'''

'\nSTEP_SIZE_TEST=test_generator.n//test_generator.batch_size\ntest_generator.reset()\npred=model.predict_generator(test_generator,\nsteps=STEP_SIZE_TEST,\nverbose=1)\n'

In [14]:
'''
STEP_SIZE_TEST=train_generator.n//train_generator.batch_size
train_generator.reset()
pred=model.predict_generator(train_generator,
    steps=STEP_SIZE_TEST,
    verbose=1)
'''

'\nSTEP_SIZE_TEST=train_generator.n//train_generator.batch_size\ntrain_generator.reset()\npred=model.predict_generator(train_generator,\n    steps=STEP_SIZE_TEST,\n    verbose=1)\n'